# Deploying a scikit-learn model on Verta

Within Verta, a "Model" can be any arbitrary function: a traditional ML model (e.g., sklearn, PyTorch, TF, etc); a function (e.g., squaring a number, making a DB function etc.); or a mixture of the above (e.g., pre-processing code, a DB call, and then a model application.) See more [here](https://docs.verta.ai/verta/registry/concepts).

This notebook provides an example of how to deploy a scikit-learn model on Verta as a Verta Standard Model either via  convenience functions or by extending [VertaModelBase](https://verta.readthedocs.io/en/master/_autogen/verta.registry.VertaModelBase.html?highlight=VertaModelBase#verta.registry.VertaModelBase).

## 0. Imports

In [1]:
# restart your notebook if prompted on Colab
!python -m pip install verta
!python -m pip install wget

In [2]:
from __future__ import print_function

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import os
import time

import numpy as np
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

### 0.1 Verta import and setup

In [3]:
import os

# Ensure credentials are set up, if not, use below
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST'] = 

from verta import Client

PROJECT_NAME = "Census"
EXPERIMENT_NAME = "sklearn"
client = Client()
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

## 1. Model Training

### 1.1 Load training data

In [4]:
import wget

train_data_url = "http://s3.amazonaws.com/verta-starter/census-train.csv"
train_data_filename = wget.detect_filename(train_data_url)
if not os.path.isfile(train_data_filename):
    wget.download(train_data_url)

test_data_url = "http://s3.amazonaws.com/verta-starter/census-test.csv"
test_data_filename = wget.detect_filename(test_data_url)
if not os.path.isfile(test_data_filename):
    wget.download(test_data_url)

In [5]:
from verta.dataset import Path

dataset = client.set_dataset(name="Census Income")
dataset_version = dataset.create_version(
    Path(train_data_filename),  # could also be S3("s3://verta-starter")
)

In [6]:
df_train = pd.read_csv(train_data_filename)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv(test_data_filename)
X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:, -1]


df_train.head()

#### Define hyperparams

In [7]:
hyperparam_candidates = {
    'C': [1e-6, 1e-4],
    'solver': ['lbfgs'],
    'max_iter': [15, 28],
}
hyperparam_sets = [dict(zip(hyperparam_candidates.keys(), values))
                   for values
                   in itertools.product(*hyperparam_candidates.values())]

### 1.3 Train/test code

In [8]:
def run_experiment(hyperparams):
    # create object to track experiment run
    run = client.set_experiment_run()
    
    # create validation split
    (X_val_train, X_val_test,
     y_val_train, y_val_test) = model_selection.train_test_split(X_train, y_train,
                                                                 test_size=0.2,
                                                                 shuffle=True)

    # log hyperparameters
    run.log_hyperparameters(hyperparams)
    print(hyperparams, end=' ')
    
    # create and train model
    model = linear_model.LogisticRegression(**hyperparams)
    model.fit(X_train, y_train)
    
    # calculate and log validation accuracy
    val_acc = model.score(X_val_test, y_val_test)
    run.log_metric("val_acc", val_acc)
    print("Validation accuracy: {:.4f}".format(val_acc))
    
    run.log_dataset_version("train", dataset_version)
    
    run.log_code()
    
# NOTE: run_experiment() could also be defined in a module, and executed in parallel
for hyperparams in hyperparam_sets:
    run_experiment(hyperparams)

In [9]:
best_run = expt.expt_runs.sort("metrics.val_acc", descending=True)[0]
print("Validation Accuracy: {:.4f}".format(best_run.get_metric("val_acc")))

best_hyperparams = best_run.get_hyperparameters()
print("Hyperparameters: {}".format(best_hyperparams))

In [10]:
model = linear_model.LogisticRegression(multi_class='auto', **best_hyperparams)
model.fit(X_train, y_train)
train_acc = model.score(X_train, y_train)
print("Training accuracy: {:.4f}".format(train_acc))

## 2. Register Model for deployment

In [11]:
registered_model = client.get_or_create_registered_model(
    name="census-sklearn", labels=["tabular", "sklearn"])

### Register a serialized version of the model using the VertaModelBase

In [12]:
import cloudpickle
with open("model.pkl", "wb") as f:
    cloudpickle.dump(model, f)

In [13]:
from verta.registry import VertaModelBase, verify_io

class CensusIncomeClassifier(VertaModelBase):
    def __init__(self, artifacts):
        self.logreg = cloudpickle.load(open(artifacts["serialized_model"], "rb"))
        
    @verify_io
    def predict(self, batch_input):
        results = []
        for one_input in batch_input:
            results.extend(self.logreg.predict([one_input]).tolist())
        return results

In [14]:
artifacts_dict = {"serialized_model" : "model.pkl"}
clf = CensusIncomeClassifier(artifacts_dict)
clf.predict(X_test.values.tolist()[:5])

In [15]:
from verta.environment import Python

model_version = registered_model.create_standard_model(
    model_cls=CensusIncomeClassifier,
    environment=Python(requirements=["scikit-learn", "pandas"]),
    code_dependencies=[],
    artifacts=artifacts_dict,
    model_api=ModelAPI(X_train, y_train),
    name="v1",
)

## 3. Deploy model to endpoint

In [16]:
census_endpoint = client.get_or_create_endpoint("census-sklearn")
census_endpoint.update(model_version, wait=True)

In [17]:
deployed_model = census_endpoint.get_deployed_model()
deployed_model.predict(X_test.values.tolist()[:5])

In [18]:
census_endpoint.delete()

---